# 데이터 전처리

In [3]:
import pandas as pd 
import warnings 
warnings.filterwarnings('ignore')


# 파일 불러오기
all = pd.read_excel("./엑셀/Join_data.xlsx", index_col=0)

# 필요한 칼럼만 남기기
all2 = all[["DateTime", "종가_ex", "1Y_Mid_irs", "10Y_Mid_irs", "1Y_Mid_crs", "국고1년", "통안364일", "Mid_ndf", "M1_스왑포인트"]]

# 0값 삭제
index1 = all2[all2["1Y_Mid_crs"] == 0].index
all2.drop(index1, inplace=True)
index2 = all2[all2["통안364일"] == 0].index
all2.drop(index2, inplace=True)
all2.reset_index(drop=True, inplace=True)

# 전일비, 전일종가 칼럼 생성
all2["전일비_ex"] = (all2["종가_ex"] - all2["종가_ex"].shift(1)) / all2["종가_ex"].shift(1)
all2["전일비_1Y_irs"] = (all2["1Y_Mid_irs"] - all2["1Y_Mid_irs"].shift(1)) / all2["1Y_Mid_irs"].shift(1)
all2["전일비_10Y_irs"] = (all2["10Y_Mid_irs"] - all2["10Y_Mid_irs"].shift(1)) / all2["10Y_Mid_irs"].shift(1)
all2["전일비_1Y_crs"] = (all2["1Y_Mid_crs"] - all2["1Y_Mid_crs"].shift(1)) / all2["1Y_Mid_crs"].shift(1)
all2["전일비_국고"] = (all2["국고1년"] - all2["국고1년"].shift(1)) / all2["국고1년"].shift(1)
all2["전일비_통안"] = (all2["통안364일"] - all2["통안364일"].shift(1)) / all2["통안364일"].shift(1)
all2["전일비_ndf"] = (all2["Mid_ndf"] - all2["Mid_ndf"].shift(1)) / all2["Mid_ndf"].shift(1)
all2["M1_스왑포인트(원)"] = all2["M1_스왑포인트"]/100 
all2["전일종가_ex"] = all2["종가_ex"].shift(1)

# Nan 행 지우고 인덱스 초기화
all2.drop([0], axis=0, inplace=True)
all2.reset_index(drop=True, inplace=True)

# 전일비 칼럼만 남기기
all_rate = all2.copy()
all_rate = all_rate[["DateTime","종가_ex", "전일비_1Y_irs", "전일비_10Y_irs", "전일비_1Y_crs", "전일비_국고", "전일비_통안", "전일비_ndf", "M1_스왑포인트(원)", "전일종가_ex"]]

# DateTime을 인덱스로 바꿔주기
all_rate = all_rate.set_index('DateTime')
all_rate.head(2)

,종가_ex,전일비_1Y_irs,전일비_10Y_irs,전일비_1Y_crs,전일비_국고,전일비_통안,전일비_ndf,M1_스왑포인트(원),전일종가_ex
DateTime,,,,,,,,,
2012-08-02,1131.7,-0.010526,-0.017182,-0.004785,-0.010638,-0.010714,0.001546,2.5,1126.5
2012-08-03,1134.8,-0.010638,-0.006993,-0.004808,-0.007168,-0.007220,0.003527,2.4,1131.7


# 그냥 회귀

In [4]:
x = all_rate[[ "전일비_1Y_crs", "전일비_ndf",  "전일종가_ex"]]
y = all_rate[["종가_ex"]]

In [5]:
import statsmodels.api as sm

feature_add = sm.add_constant(x, has_constant='add')

# sm OLS 적합
model = sm.OLS(y , feature_add)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  종가_ex   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 7.701e+04
Date:                Thu, 18 Aug 2022   Prob (F-statistic):               0.00
Time:                        20:44:38   Log-Likelihood:                -7735.5
No. Observations:                2449   AIC:                         1.548e+04
Df Residuals:                    2445   BIC:                         1.550e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.0556      2.355      1.722      0.085      -0.563       8.674
전일비_1Y_crs     0.9086      0.260      3.490      0.000       0.398       1.419
전일비_ndf       15.1731      4.215      3.599      0.000       6.907      23.439
전일종가_ex        0.9965      0.002    480.643      0.000       0.992       1.001
==============================================================================
Omnibus:                      150.200   Durbin-Watson:                   2.144
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              650.671
Skew:                           0.037   Prob(JB):                    5.11e-142
Kurtosis:                       5.524   Cond. No.                     4.16e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.16e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [8]:
import numpy as np 
# import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)
from sklearn.linear_model import LinearRegression

mlr = LinearRegression()
mlr.fit(x_train, y_train)

y_pred = mlr.predict(x_test)

from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print('MSE : {0:}, RMSE : {1: .3f}'.format(mse, rmse))
print('Variance score : {0: .3f}'.format(r2_score(y_test, y_pred)))


MSE : 34.63326953988273, RMSE :  5.885
Variance score :  0.988
